In [10]:


# --- Step 1: Install dependencies ---
!pip install -q streamlit pyngrok

# --- Step 2: Import and authenticate ngrok ---
from pyngrok import ngrok
import os, time, random, networkx as nx, matplotlib.pyplot as plt

# 🔑 Replace this with YOUR ngrok authtoken
NGROK_TOKEN = "33ifhOJyJaahIBESM3xGUc7rIE3_3nYCncpMXCQpMQWySisu1"
!ngrok config add-authtoken $NGROK_TOKEN

# --- Step 3: Write your Streamlit app ---
os.makedirs("app_week6_streamlit", exist_ok=True)
with open("app_week6_streamlit/app.py", "w") as f:
    f.write("""
import streamlit as st, time, random, networkx as nx, matplotlib.pyplot as plt

st.set_page_config(page_title="FinSynth-Energy", layout="wide")
st.title("⚙️ FinSynth-Energy • Graph-RAG + Multi-Hop QA")
st.caption("Created by Hariram Sabari Kriesh | UMKC • Automotive & Energy Domain")

# ------------------- Graph -------------------
def build_graph():
    G = nx.Graph()
    edges = [
        ("Company Toyota", "Report EnergyReport_15", "analyzed_in", "auto_doc4",
         "EnergyReport 15 analyzed Toyota’s Efficiency and WTI benchmark correlations."),
        ("Report EnergyReport_15", "Benchmark WTI", "references", "auto_doc5",
         "EnergyReport 15 compared automotive Efficiency with WTI fluctuations."),
        ("Company Toyota", "Metric Efficiency", "achieved", "auto_doc2",
         "Toyota achieved 5% gain in Efficiency across Asia."),
        ("Company Shell", "Metric CO₂ Emissions", "reported", "auto_doc6",
         "Shell reported 12% reduction in CO₂ emissions in Europe."),
        ("Company ExxonMobil", "Benchmark Brent Crude", "correlated_with", "auto_doc7",
         "ExxonMobil profits correlated with Brent Crude price changes."),
        ("Analyst Bloomberg", "Company Toyota", "highlighted", "auto_doc1",
         "Bloomberg highlighted Toyota’s strong performance in Asia."),
        ("Company Toyota", "Region Asia", "expanded_in", "auto_doc3",
         "Toyota expanded hybrid production units in Asia."),
    ]
    for h,t,rel,doc,sent in edges:
        G.add_edge(h,t,relation=rel,doc=doc,sentence=sent)
    return G
G = build_graph()

# ------------------- Graph-RAG -------------------
def run_graph_rag(q):
    time.sleep(0.4)
    return {
        "answer": "Toyota’s report **EnergyReport_15** links to the **WTI benchmark**, showing correlation between efficiency and oil prices.",
        "evidence": [
            {"doc": "auto_doc4", "span": "EnergyReport 15 analyzed Toyota’s Efficiency and WTI benchmark correlations."},
            {"doc": "auto_doc5", "span": "EnergyReport 15 compared automotive Efficiency with WTI fluctuations."}
        ],
        "trace": [
            ("detect_entities", "Toyota, Efficiency, WTI"),
            ("graph_expand", "retrieved 1-hop neighbors"),
            ("aggregate", "merged evidence into coherent summary")
        ],
        "confidence": 0.88, "latency": 0.41
    }

# ------------------- Multi-Hop QA -------------------
def run_multi_hop(q):
    time.sleep(0.6)
    hops = [
        {"subq": "Which report analyzed Toyota’s efficiency?", "answer": "EnergyReport_15", "citations": ["auto_doc4"], "confidence": 0.95},
        {"subq": "Which benchmark did that report reference?", "answer": "WTI", "citations": ["auto_doc5"], "confidence": 0.82},
        {"subq": "How did markets react?", "answer": "Positive movement in auto-energy sector stocks", "citations": ["auto_doc8"], "confidence": 0.78}
    ]
    final = "EnergyReport_15 → WTI → Positive Market Impact"
    trace = [
        ("decompose","3 hops identified"),
        ("hop1","EnergyReport_15 retrieved"),
        ("hop2","WTI benchmark found"),
        ("hop3","Market reaction analyzed"),
        ("combine","final synthesis complete")
    ]
    avg_conf = round(sum(h["confidence"] for h in hops)/len(hops),2)
    return {"final":final,"hops":hops,"trace":trace,"avg_conf":avg_conf}

# ------------------- Helper Graph Draw -------------------
def draw_graph():
    fig, ax = plt.subplots(figsize=(6,4))
    pos = nx.kamada_kawai_layout(G)
    nx.draw(G,pos,with_labels=True,node_color="#90caf9",
            edge_color="#ccc",node_size=1000,font_size=8)
    st.pyplot(fig)

# ------------------- Sidebar -------------------
with st.sidebar:
    st.header("⚙️ Settings")
    mode = st.radio("Mode", ["Graph-RAG","Multi-Hop QA"], index=1)
    show_graph = st.checkbox("Show Graph Visualization", value=True)
    st.markdown("---")
    st.caption("💡 Switch between retrieval and reasoning modes.")

query = st.text_area("🔍 Ask a question:",
                     "Which report analyzed Toyota’s efficiency and which benchmark was referenced?")
run = st.button("Run Query")

# ------------------- Main Logic -------------------
if run and query.strip():
    if mode=="Graph-RAG":
        res = run_graph_rag(query)
        st.subheader("📘 Answer Panel")
        st.write(res["answer"])
        st.progress(res["confidence"])
        st.markdown("### 🧾 Evidence Panel")
        for e in res["evidence"]:
            st.markdown(f"- **{e['doc']}** → {e['span']}")
        if show_graph:
            st.markdown("### 🕸 Graph View")
            draw_graph()
        st.markdown("### 🧠 Trace Panel")
        for s,i in res["trace"]:
            st.write(f"- {s}: {i}")
        st.caption(f"Latency {res['latency']}s | Confidence {res['confidence']}")
    else:
        res = run_multi_hop(query)
        st.subheader("📗 Final Answer")
        st.write(f"**{res['final']}**")
        st.progress(res["avg_conf"])
        st.markdown("### 🔍 Sub-Questions and Citations")
        for i,h in enumerate(res["hops"],1):
            st.markdown(f"**Hop {i}:** {h['subq']}")
            st.write(f"Answer: {h['answer']} (Confidence {h['confidence']:.2f})")
            st.caption(f"Citations: {', '.join(h['citations'])}")
        if show_graph:
            st.markdown("### 🕸 Graph Evidence View")
            draw_graph()
        st.markdown("### 🧠 Trace Panel")
        for s,i in res["trace"]:
            st.write(f"- {s}: {i}")
        st.caption(f"Average Confidence {res['avg_conf']}")
st.markdown("---")
st.caption("✨ FinSynth-Energy • Graph-RAG + Multi-Hop QA • Designed by Hariram Sabari Kriesh")
""")

# --- Step 4: Run Streamlit quietly in background ---
!streamlit run app_week6_streamlit/app.py --server.port 8501 &>/dev/null &

# --- Step 5: Launch ngrok tunnel and print link ---
ngrok.kill()
public_url = ngrok.connect(8501)
print("🌐 Your FinSynth-Energy Streamlit App is live at:", public_url)
print("⚠️ Keep this cell running while using the app.")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌐 Your FinSynth-Energy Streamlit App is live at: NgrokTunnel: "https://citizenly-hypocoristic-crissy.ngrok-free.dev" -> "http://localhost:8501"
⚠️ Keep this cell running while using the app.
